# EEEN30131 Power System Analysis: Week 05 - Examples

***&copy; 2024 Martínez Ceseña — University of Manchester, UK***

This notebook is dedicated to providing examples covering frequency regulation and generation control. The specific tools presented here have been developed in other notebooks, expecially in `EEEN30131_Week05`.

The use of the notebooks is optional and will not be marked. That said, you are strongly encouraged to play with the tools and examples, as such activities will better prepare you for the exams.

## List of contents

- [Accuracy of the DC power flow](#Accuracy-of-the-DC-power-flow)
- [Primary frequency regulation](#Primary-frequency-regulation)
- [Secondary frequency regulation](#Secondary-frequency-regulation)

## Before we begin

Before we begin: 
- Make sure to review the asynchronous materials provided in blackboard for EEEN30131 Week 4 - Frequency regulation 
- If you have any questions, please post them in the discussion boards or, if that is not possible, send an email to alex.martinezcesena@manchester.ac.uk

This notebook builds on the materials that were presented during the last week. Therefore, before we begin: 
- Make sure to review the asynchronous materials provided in blackboard for EEEN30131:
  - Week 4 - Frequency regulation
  - Week 5 - Interconnected systems

Also, the interconnected models are based on a simplified approach to model power flows. Therefore, it is also recommended to review our previous lectures on power flow analysis:
  - Week 1 - Nodal analysis 
  - Week 2 - Power Flow Formulation 
  - Week 3 - Newton Raphson

- If you have any questions, please post them in the discussion boards or, if that is not possible, send an email to alex.martinezcesena@manchester.ac.uk

If this data notebook is being used in Jupyter lite, the folders where the python code that supports this notebook are stored, have to be enabled.

In [ ]:
from pathlib import Path
if Path.cwd().drive == '':
    a_dir = Path("dir")
    a_dir.mkdir(exist_ok=True)

The following general python libraries are needed:

In [ ]:
import numpy
try:
    import ipywidgets as widgets
except:
    import micropip
    await micropip.install('ipywidgets')
    import ipywidgets as widgets
from ipywidgets import interact

The notebook borrows several tools developed in previous weeks, so we need to import them here:

In [1]:
from Code.Wk1_EEEN30131 import get_Ybus
from Code.Wk2_EEEN30131 import get_Bus_Type, develop_PF_Equations
from Code.Wk3_EEEN30131 import Newtons_Method, Visualize_Elec
from Code.Wk4_EEEN30131 import get_Primary_Response, get_pu, get_Δw, get_step
from Code.Wk5_EEEN30131 import run_DCPF, Visualize_DC, get_Tie_Data, get_TieLine, print_TieLine
from Code.Wk5_EEEN30131 import interconnected_frequency_primary, plot_Δw, interconnected_power
from Code.Wk5_EEEN30131 import plot_Ppu, interconnected_model, interconnected_frequency_secondary

[Back to top](#EEEN30131-Power-System-Analysis:-Week-05---Examples)

## Accuracy of the DC power flow

The DC model is generally accurate for the analysis of transmission networks where voltages are close to 1 pu and the line resistances are negligible.

<img src="Figures/Week02_3Bus_Empty.png" alt="Fig01" class="bg-primary" width="500px">

Check how the accuracy of the model changes if you increase the line resistances.

In [3]:
@interact
def Bus3(P2 = widgets.FloatSlider(min=0.0,max=5,step=0.1,value=1.5,description='P2 (pu)'),
         Q2 = widgets.FloatSlider(min=0.0,max=5,step=0.1,value=0.4,description='Q2 (pu)'),
         R12 = widgets.FloatSlider(min=0.0,max=0.05,step=0.01,value=0.0,description='R1,2 (pu)'),
         X12 = widgets.FloatSlider(min=0.0,max=0.05,step=0.01,value=0.01,description='X1,2 (pu)'),
         R13 = widgets.FloatSlider(min=0.0,max=0.05,step=0.01,value=0.0,description='R1,3 (pu)'),
         X13 = widgets.FloatSlider(min=0.0,max=0.05,step=0.01,value=0.02,description='X1,3 (pu)'),
         R23 = widgets.FloatSlider(min=0.0,max=0.05,step=0.01,value=0.0,description='R2,3 (pu)'),
         X23 = widgets.FloatSlider(min=0.0,max=0.05,step=0.01,value=0.03,description='X2,3 (pu)')):

    Connectivity = [
        [1, 2, complex(R12, X12)],
        [1, 3, complex(R13, X13)],
        [2, 3, complex(R23, X23)]
    ]
    Load = [
        [2, complex(P2, Q2)]
    ]
    Generator = [
        {'Bus':1, 'V':1, '𝜃':0 },
        {'Bus':3, 'P':1, 'Q':0.25}
    ]

    Succes, 𝜃, P = run_DCPF(Connectivity, Load, Generator)
    Visualize_DC(Connectivity, 𝜃, P, Succes)

    # From Week 01
    Ybus = get_Ybus(Connectivity, True, False)

    # From Week 02
    P_Data, Q_Data = develop_PF_Equations(Load, Generator, Ybus, True, False)
    Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)

    # From Week 03
    V_All, 𝜃_All, Threshold, Succes = Newtons_Method(P_Data, Q_Data, Bus_Data, Bus_Type, Generator, 0)
    Visualize_Elec(Connectivity, V_All, 𝜃_All, Succes)

interactive(children=(FloatSlider(value=1.5, description='P2 (pu)', max=5.0), FloatSlider(value=0.4, descripti…

[Back to top](#EEEN30131-Power-System-Analysis:-Week-05---Examples)

## Primary frequency regulation

In the context of primary frequency regulation:
> What happens when the contingency occurs in different areas?

> What are the impacts of different contingencies?

In [4]:
@interact
def Bus3(Area = widgets.FloatSlider(min=1,max=2,step=1,value=1),
         Contingency = widgets.FloatSlider(min=-500,max=500,step=5,value=100,
                                           description='ΔP (MW)', continuous_update=False)):
    Dist = {
        'Area': Area,
        'Magnitude': Contingency,  # Positive: Generation is higher than load
        'Flow': 0  # From area 1 to 2 by default
    }
    Gen = [
        {'Area':1, 'Capacity': 1000, 'R':0.0500, 'Units': 'pu', 'Output': 500},
        {'Area':2, 'Capacity': 1000, 'R':0.0625, 'Units': 'pu', 'Output': 500}
    ]
    Load = [
        {'D':0.6, 'Units': 'pu'},
        {'D':0.9, 'Units': 'pu'}
    ]
    Base = 1000
    F = 50
    T = 30

    # System data
    M = [10, 6]
    Tch = [0.5, 0.6]
    Kg = [100, 53.3333]
    Tl = 2

    Disturbance, Generators, Damping = get_pu(Dist, Gen, Load, Base, F)
    Δw, _, _ = get_Δw(Disturbance, Generators, Damping, F)

    # Print tie line
    R, D = get_Tie_Data(Dist, Generators, Damping, Kg)
    Tg = [1/Kg[0]/R[0], 1/Kg[1]/R[1]]
    ΔPtie, ΔPtie_pu, Ptie, a1, a2 = get_TieLine(Δw, Disturbance, Generators, Damping, Base)
    print_TieLine(ΔPtie, ΔPtie_pu, Ptie, a1, a2)

    # Create input signals
    t = numpy.linspace(0,T,1000)
    ΔPL = get_step(t, 1, Disturbance['Magnitude'])

    # Model frequency response
    Δw1, Δw2, mod = interconnected_frequency_primary(Dist['Area'], R, D, M, Tch, Kg, Tg, Tl, t, ΔPL)

    # Model power response
    Tie, P1, P2 = interconnected_power(mod, t, Δw1, Δw2)

    # Plot frequency and power response
    if Area == 1:
        plot_Δw(t, Δw1, Δw2)
        plot_Ppu(t, Tie, P1, P2)
        print('Δw: %.4f pu after %d seconds'%(Δw1[-1], T))
    else:
        plot_Δw(t, Δw2, Δw1)
        plot_Ppu(t, Tie, P2, P1)
        print('Δw: %.4f pu after %d seconds'%(Δw2[-1], T))
    print('Tie: %.4f pu after %d seconds'%(Tie[-1], T))

interactive(children=(FloatSlider(value=1.0, description='Area', max=2.0, min=1.0, step=1.0), FloatSlider(valu…

[Back to top](#EEEN30131-Power-System-Analysis:-Week-05---Examples)

> Compare the steady-state and dynamic results

In [5]:
@interact
def Bus3(Area = widgets.FloatSlider(min=1,max=2,step=1,value=1),
         Contingency = widgets.FloatSlider(min=-500,max=500,step=5,value=100,
                                           description='ΔP (MW)', continuous_update=False)):
    Dist = {
        'Area': Area,
        'Magnitude': Contingency,  # Positive: Generation is higher than load
        'Flow': 0  # From area 1 to 2 by default
    }
    Gen = [
        {'Area':1, 'Capacity': 1000, 'R':0.0500, 'Units': 'pu', 'Output': 500},
        {'Area':2, 'Capacity': 1000, 'R':0.0625, 'Units': 'pu', 'Output': 500}
    ]
    Load = [
        {'D':0.6, 'Units': 'pu'},
        {'D':0.9, 'Units': 'pu'}
    ]
    Base = 1000
    F = 50
    T = 30
    get_Primary_Response(Dist, Gen, Load, Base, F)

    # System data
    M = [10, 6]
    Tch = [0.5, 0.6]
    Kg = [100, 53.3333]
    Tl = 2

    Disturbance, Generators, Damping = get_pu(Dist, Gen, Load, Base, F)
    Δw, _, _ = get_Δw(Disturbance, Generators, Damping, F)

    # Print tie line
    R, D = get_Tie_Data(Dist, Generators, Damping, Kg)
    Tg = [1/Kg[0]/R[0], 1/Kg[1]/R[1]]
    ΔPtie, ΔPtie_pu, Ptie, a1, a2 = get_TieLine(Δw, Disturbance, Generators, Damping, Base)
    print_TieLine(ΔPtie, ΔPtie_pu, Ptie, a1, a2)

    # Create input signals
    t = numpy.linspace(0,T,1000)
    ΔPL = get_step(t, 1, Disturbance['Magnitude'])

    # Model frequency response
    Δw1, Δw2, mod = interconnected_frequency_primary(Dist['Area'], R, D, M, Tch, Kg, Tg, Tl, t, ΔPL)

    # Model power response
    Tie, P1, P2 = interconnected_power(mod, t, Δw1, Δw2)

    # Plot frequency and power response
    if Area == 1:
        plot_Δw(t, Δw1, Δw2)
        plot_Ppu(t, Tie, P1, P2)
        print('Δw: %.4f pu after %d seconds'%(Δw1[-1], T))
    else:
        plot_Δw(t, Δw2, Δw1)
        plot_Ppu(t, Tie, P2, P1)
        print('Δw: %.4f pu after %d seconds'%(Δw2[-1], T))
    print('Tie: %.4f pu after %d seconds'%(Tie[-1], T))

interactive(children=(FloatSlider(value=1.0, description='Area', max=2.0, min=1.0, step=1.0), FloatSlider(valu…

[Back to top](#EEEN30131-Power-System-Analysis:-Week-05---Examples)

## Secondary frequency regulation

In the context of secondary frequency regulation:
> What happens when the contingency occurs in different areas?

> What are the impacts of different contingencies?

In [6]:
@interact
def Bus3(Area = widgets.FloatSlider(min=1,max=2,step=1,value=1),
         Contingency = widgets.FloatSlider(min=-500,max=500,step=5,value=100,
                                           description='ΔP (MW)', continuous_update=False)):
    R = [0.05, 0.0625]
    Beta = [20.6, 16.9]
    D = [0.6, 0.9]
    M = [10, 6]
    Tch = [0.5, 0.6]
    Kt = [0.3, 0.3]
    Kg = [100, 53.3333]
    Tg = [1/Kg[0]/R[0], 1/Kg[1]/R[1]]
    Tl = 2

    Base = 1000
    ΔPL = Contingency/Base
    T = 30
    mod = interconnected_model(R, D, M, Tch, Kg, Tg, Tl, Beta, Kt)
    t, Δw1, Δw2 = interconnected_frequency_secondary(mod, Area, ΔPL, T)
    Tie, P1, P2 = interconnected_power(mod, t, Δw1, Δw2)
    if Area == 1:
        plot_Δw(t, Δw1, Δw2)
        plot_Ppu(t, Tie, P1, P2)
    else:
        plot_Δw(t, Δw2, Δw1)
        plot_Ppu(t, Tie, P2, P1)

interactive(children=(FloatSlider(value=1.0, description='Area', max=2.0, min=1.0, step=1.0), FloatSlider(valu…

[Back to top](#EEEN30131-Power-System-Analysis:-Week-05---Examples)